In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import StandardScaler

# Part 2 {-}
See https://exoplanetarchive.ipac.caltech.edu (for context/source). We are using the Composite Planetary Systems dataset

# Q2a {-}
Say we’re interested in modeling the radius of exoplanets in kilometers, which is named as `pl_rade` in the data. Note that the variable `pl_rade` captures the radius of each plant as a proportion of Earth’s radius, which is approximately 6,378.1370 km. 

Develop a linear regression model to predict `pl_rade` using all the variables in *train_CompositePlanetarySystems.csv* except `pl_name`, `disc_facility` and `disc_locale`. Find the RMSE (Root mean squared error) of the model on *test1_CompositePlanetarySystems.csv* and *test2_CompositePlanetarySystems.csv*. 

*(4 points for code)*

In [31]:
#Importing data
train = pd.read_csv('train_CompositePlanetarySystems.csv')
test1 = pd.read_csv('test1_CompositePlanetarySystems.csv')
test2 = pd.read_csv('test2_CompositePlanetarySystems.csv')

In [32]:
#Model
predictors = '+'.join(list(train.drop(columns = ['pl_name', 'disc_facility', 'disc_locale', 'pl_rade']).columns))
model = sm.ols('pl_rade~'+predictors, data = train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                pl_rade   R-squared:                       0.724
Model:                            OLS   Adj. R-squared:                  0.719
Method:                 Least Squares   F-statistic:                     124.8
Date:                Mon, 07 Mar 2022   Prob (F-statistic):          4.83e-249
Time:                        21:43:16   Log-Likelihood:                -2358.9
No. Observations:                 970   AIC:                             4760.
Df Residuals:                     949   BIC:                             4862.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         370.5206     48.434      7.650      0.000     275.470     465.572
sy_snum             0.0252      0.297      0.085      0.932      -0.558       0.608
sy_pnum            -0.7530      0.082     -9.157      0.000      -0.914      -0.592
cb_flag            -1.9300      2.001     -0.964      0.335      -5.857       1.997
disc_year          -0.1781      0.024     -7.480      0.000      -0.225      -0.131
rv_flag             7.2423      0.265     27.343      0.000       6.722       7.762
pul_flag        -1.086e-09   2.46e-10     -4.411      0.000   -1.57e-09   -6.03e-10
ptv_flag            3.5059      4.501      0.779      0.436      -5.327      12.339
tran_flag           1.1365      0.352      3.232      0.001       0.446       1.827
ast_flag            1.5613      2.059      0.758      0.448      -2.479       5.601
obm_flag           -0.9951      1.149     -0.866      0.387      -3.250       1.260
micro_flag       1.405e-11   3.18e-12      4.417      0.000    7.81e-12    2.03e-11
etv_flag            4.0156      5.375      0.747      0.455      -6.533      14.564
ima_flag        -2362.1345    540.655     -4.369      0.000   -3423.151   -1301.118
pl_controv_flag    -4.4229      2.569     -1.722      0.085      -9.464       0.618
pl_orbper           0.0003   6.73e-05      4.404      0.000       0.000       0.000
pl_dens            -0.0088      0.009     -0.964      0.335      -0.027       0.009
ttv_flag            0.8794      0.362      2.432      0.015       0.170       1.589
st_teff             0.0005      0.000      2.860      0.004       0.000       0.001
st_rad             -0.1803      0.057     -3.182      0.002      -0.291      -0.069
st_mass             3.3632      0.580      5.803      0.000       2.226       4.501
st_lum             -1.1531      0.448     -2.572      0.010      -2.033      -0.273
st_logg            -3.2476      0.677     -4.796      0.000      -4.576      -1.919
==============================================================================
Omnibus:                       45.899   Durbin-Watson:                   1.928
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              141.548
Skew:                           0.088   Prob(JB):                     1.83e-31
Kurtosis:                       4.863   Cond. No.                     8.64e+23
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.66e-35. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [33]:
rmse1 = np.sqrt(((test1.pl_rade - model.predict(test1))**2).mean())
rmse2 = np.sqrt(((test2.pl_rade - model.predict(test2))**2).mean())
print(f'The RMSE for test 1 is {rmse1}. The RMSE for test 2 is {rmse2}')

The RMSE for test 1 is 127.36595496362591. The RMSE for test 2 is 202.66277572552613


# Q2b {-}
Develop a ridge regression model to predict `pl_rade` using all the variables in *train_CompositePlanetarySystems.csv* except `pl_name`, `disc_facility` and `disc_locale`. What is the optimal value of the tuning parameter $\lambda$?

**Hint:** You may use the following grid of lambda values to find the optimal $\lambda$: 
`alphas = 10**np.linspace(2,0.5,200)*0.5`

Remember to standardize data before fitting the ridge regression model


*(5 points for code)*

In [34]:
# Standardizing data
scaler = StandardScaler()
y = train['pl_rade']
X = train.drop(columns = ['pl_name', 'disc_facility', 'disc_locale', 'pl_rade'])
scaler.fit(X)
Xstd = scaler.transform(X)

# Tuning Parameters
alphas = 10**np.linspace(2,0.5,200)*0.5

# Ridge: Finding Optimal lambda
ridgecv = RidgeCV(alphas = alphas,store_cv_values=True)
ridgecv.fit(Xstd, y)

#Optimal value of the tuning parameter - lamda
ridgecv.alpha_

3.221181754360687

# Q2c {-}
Use the optimal value of $\lambda$ found in the previous question to develop a ridge regression model. What is the RMSE of the model on *test1_CompositePlanetarySystems.csv* and *test2_CompositePlanetarySystems.csv*?

*(5 points for code)*

In [35]:
#Standardizing tests
X_test1 = test1.drop(columns = ['pl_name', 'disc_facility', 'disc_locale', 'pl_rade'])
X_test2 = test2.drop(columns = ['pl_name', 'disc_facility', 'disc_locale', 'pl_rade'])
Xtest1_std = scaler.transform(X_test1)
Xtest2_std = scaler.transform(X_test2)

ridge = Ridge(alpha = ridgecv.alpha_)
ridge.fit(Xstd, y)
pred = ridge.predict(Xtest1_std)
rmse_1 = np.sqrt(((pred - test1.pl_rade)**2).mean())
print('test 1 RMSE:', rmse_1)

pred=ridge.predict(Xtest2_std)
rmse_2 = np.sqrt(((pred - test2.pl_rade)**2).mean())
print('test 2 RMSE:', rmse_2)

test 1 RMSE: 3.2187088791485707
test 2 RMSE: 2.989865167853597


# Q2d {-}
Note that ridge regression has a much lower RMSE on test datasets as compared to Ordinary least squares (OLS) regression. Shrinking the coefficients has improved the model fit. Appreciate it. Which are the top two predictors for which the coefficients have shrunk the most? 

To answer this question, find the ridge regression estimates for $\lambda = 10^{-10}$. Treat these estimates as OLS estimates and find the predictors for which these estimates have shrunk the most in the model developed in 2c.

*(4 points for code, 1 point for answer)*

In [36]:
ridge_lambda = Ridge(alpha = 10**(-10))
ridge_lambda.fit(Xstd, y)
coefs_lambda = pd.Series(ridge_lambda.coef_, index = X.columns)
coefs_lambda.sort_values()

ima_flag          -75.804463
st_logg            -1.383991
sy_pnum            -0.876263
st_lum             -0.812958
disc_year          -0.780497
st_rad             -0.513427
pl_controv_flag    -0.200628
cb_flag            -0.123682
pl_dens            -0.086940
obm_flag           -0.084228
micro_flag          0.000000
pul_flag            0.000000
sy_snum             0.008191
ast_flag            0.070820
ptv_flag            0.112509
etv_flag            0.182152
ttv_flag            0.229052
tran_flag           0.453724
st_teff             0.954692
st_mass             1.055440
rv_flag             3.469238
pl_orbper          76.418631
dtype: float64

In [37]:
coefs = pd.Series(ridge.coef_, index = X.columns)
coefs.sort_values()

st_logg           -1.270665
sy_pnum           -0.888378
disc_year         -0.717445
st_lum            -0.696500
st_rad            -0.522443
pl_controv_flag   -0.209565
cb_flag           -0.119854
obm_flag          -0.081300
pl_dens           -0.073497
micro_flag         0.000000
pul_flag           0.000000
ima_flag           0.003664
sy_snum            0.016748
ptv_flag           0.123637
ast_flag           0.166958
etv_flag           0.223187
ttv_flag           0.239778
tran_flag          0.260198
pl_orbper          0.608761
st_teff            0.881772
st_mass            1.051626
rv_flag            3.456939
dtype: float64

# Q2e {-}
Why do you think the coefficients of the two variables indentified in the previous question shrunk the most?

**Hint:** VIF!

*(4 points for justification - including any code used)*

In [38]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
X = add_constant(X)
vif = pd.DataFrame()
vif["feature"] = X.columns
# calculating VIF for each feature
#vif_data["VIF"] = [variance_inflation_factor(X.values, i)
# for i in range(len(X.columns))]
for i in range(len(X.columns)):
    vif.loc[i,'VIF'] = variance_inflation_factor(X.values, i)
print(vif)

            feature            VIF
0             const  293592.094382
1           sy_snum       1.162755
2           sy_pnum       1.146094
3           cb_flag       2.058390
4         disc_year       1.362591
5           rv_flag       2.014735
6          pul_flag            NaN
7          ptv_flag       2.610996
8         tran_flag       2.466560
9          ast_flag       1.091332
10         obm_flag       1.183595
11       micro_flag            NaN
12         etv_flag       7.440160
13         ima_flag   37675.155715
14  pl_controv_flag       1.698968
15        pl_orbper   37683.517115
16          pl_dens       1.017593
17         ttv_flag       1.110245
18          st_teff      13.944718
19           st_rad       3.257504
20          st_mass       4.139998
21           st_lum      12.507339
22          st_logg      10.421923


C:\Users\nvsim\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


**Because these two variables have such an strong linear relationship, they would introduce colinearity issues into the model.**

# Q2f {-}
Develop a lasso model to predict `pl_rade` using all the variables in *train_CompositePlanetarySystems.csv* except `pl_name`, `disc_facility` and `disc_locale`. What is the optimal value of the tuning parameter $\lambda$?

**Hint:** You may use the following grid of lambda values to find the optimal $\lambda$: 
`alphas = 10**np.linspace(0,-2.5,200)*0.5`

*(4 points for code)*

In [39]:
alphas = 10**np.linspace(0,-2.5,200)*0.5
lassocv = LassoCV(alphas = alphas, cv = 10, max_iter = 100000)
lassocv.fit(Xstd, y)

#Optimal value of the tuning parameter - lamda
lassocv.alpha_

0.002051329052913595

# Q2g {-}
Use the optimal value of $\lambda$ found in the previous question to develop a lasso model. What is the RMSE of the model on *test1_CompositePlanetarySystems.csv* and *test2_CompositePlanetarySystems.csv*?

*(5 points for code)*

In [40]:
# Lasso Regression - Test 1
lasso = Lasso(alpha = lassocv.alpha_)
lasso.fit(Xstd, y)
pred=lasso.predict(Xtest1_std)
rmse_1 = np.sqrt(((pred - test1.pl_rade)**2).mean())
print('test 1 RMSE:', rmse_1)

# Lasso Regression - Test 2
pred=lasso.predict(Xtest2_std)
rmse_2 = np.sqrt(((pred - test2.pl_rade)**2).mean())
print('test 2 RMSE:', rmse2)

test 1 RMSE: 3.220582381965326
test 2 RMSE: 202.66277572552613


# Q2h {-}
Note that lasso has a much lower RMSE on test datasets as compared to Ordinary least squares (OLS) regression. Shrinking the coefficients has improved the model fit. Appreciate it. Which variables have been eliminated by lasso? 

To answer this question, find the predictors whose coefficients are 0 in the lasso model. 

*(2 points for code, 1 point for answer)*

In [41]:
X = train.drop(columns = ['pl_name', 'disc_facility', 'disc_locale', 'pl_rade'])
np.abs((pd.Series(lasso.coef_, index = X.columns))).sort_values()

micro_flag         0.000000
ima_flag           0.000000
pul_flag           0.000000
sy_snum            0.012105
pl_dens            0.071061
obm_flag           0.083005
ptv_flag           0.110936
cb_flag            0.114719
ast_flag           0.166013
etv_flag           0.197494
pl_controv_flag    0.215100
ttv_flag           0.239185
tran_flag          0.269474
st_rad             0.524817
pl_orbper          0.610153
disc_year          0.717696
st_lum             0.730514
sy_pnum            0.888101
st_teff            0.922826
st_mass            1.051955
st_logg            1.297343
rv_flag            3.472807
dtype: float64

**Ima_flag, micro_flag, and pul_flag were shrunk to 0 by the lasso.**